<a href="https://colab.research.google.com/github/Hacxmr/NLP_Project/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Text Summarization with Bias Detection in News

In [7]:
!pip install requests beautifulsoup4 pandas tqdm selenium


In [8]:
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

text = "Instead of promised 'acche din', days of debt have arrived: Congress's dig at Modi Government"
summary = summarizer(text, max_length=20, min_length=5, do_sample=False)


Device set to use cpu


In [9]:
print(summary)

[{'summary_text': "Instead of promised 'acche din', days of debt have arrived: Congress's dig"}]


In [10]:
summary = summarizer(text, max_length=30, min_length=5, do_sample=False)
print(summary[0]['summary_text'])


Your max_length is set to 30, but your input_length is only 22. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


Instead of promised 'acche din', days of debt have arrived: Congress's dig at Modi.


##Sentiment Analysis

In [11]:
sentiment_pipeline = pipeline("sentiment-analysis")
sentiment = sentiment_pipeline(summary[0]['summary_text'])
print(sentiment)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'NEGATIVE', 'score': 0.9930257797241211}]


##3. Explore BART/T5 Models

In [12]:
from transformers import pipeline
summarizer = pipeline("summarization", model="t5-base",tokenizer="t5-base")


input_text = "summarize: "+text
summary = summarizer(input_text, max_length=20, min_length=5, do_sample=False)
print(summary[0]['summary_text'])

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


instead of promised 'acche din', days of debt have arrived .


##Data Collection - Scraping

In [1]:
!pip install requests beautifulsoup4 pandas transformers


In [1]:
!pip install selenium pandas transformers beautifulsoup4


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import tempfile

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument(f"--user-data-dir={tempfile.mkdtemp()}")  # fix for profile conflict

driver = webdriver.Chrome(options=options)


In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from urllib.parse import urljoin
from transformers import pipeline

# NLP Pipelines
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
sentiment_analyzer = pipeline("sentiment-analysis")
bias_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Constants
BASE_URL = "https://www.indiatoday.in/elections"
HEADERS = {"User-Agent": "Mozilla/5.0"}
KEYWORDS = [
    "delhi election", "delhi polls", "aap", "kejriwal", "delhi mla",
    "bihar election", "bihar polls", "tejashwi", "nitish", "bjp", "rjd", "jdu",
    "lok sabha", "2024", "campaign", "manifesto", "seat sharing", "voting"
]
MAX_ARTICLES = 15

def is_delhi_election(text):
    return any(k in text.lower() for k in KEYWORDS)

def get_article_links():
    r = requests.get(BASE_URL, headers=HEADERS)
    soup = BeautifulSoup(r.text, "html.parser")

    links = []
    for a in soup.find_all("a", href=True, title=True):
        href = urljoin(BASE_URL, a["href"])
        title = a["title"].strip()
        if is_delhi_election(title):
            links.append((href, title))
    return list(set(links))

def extract_article_content(url):
    try:
        r = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(r.text, "html.parser")
        paragraphs = soup.find_all("p")
        content = " ".join(p.get_text(strip=True) for p in paragraphs if len(p.text.strip()) > 30)
        return content
    except Exception as e:
        print(f"[-] Error scraping {url}: {e}")
        return ""

def summarize_sentiment_bias(text):
    summary = summarizer(text[:1024])[0]['summary_text']
    sentiment = sentiment_analyzer(summary)[0]
    bias = bias_classifier(summary, candidate_labels=["pro-government", "anti-government", "neutral"])
    return summary, sentiment["label"], sentiment["score"], bias["labels"][0], bias["scores"][0]

def main():
    articles = get_article_links()
    print(f"[+] Found {len(articles)} Delhi election-related articles")

    data = []
    for i, (url, title) in enumerate(articles[:MAX_ARTICLES]):
        print(f"[{i+1}] Scraping: {title}")
        content = extract_article_content(url)
        if content:
            summary, sentiment, senti_score, bias, bias_score = summarize_sentiment_bias(content)
            data.append({
                "title": title,
                "url": url,
                "content": content,
                "summary": summary,
                "sentiment": sentiment,
                "sentiment_score": round(senti_score, 2),
                "bias": bias,
                "bias_score": round(bias_score, 2),
                "source": "India Today"
            })
        time.sleep(random.uniform(1, 2.5))

    df = pd.DataFrame(data)
    df.to_csv("indiatoday_elections.csv", index=False)
    print(f"[✓] Saved {len(df)} articles to indiatoday_delhi_elections.csv")

if __name__ == "__main__":
    main()


Device set to use cpu
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
Device set to use cpu


[+] Found 37 Delhi election-related articles
[1] Scraping: BJP's Mohan Singh Bisht proposes to rename Mustafabad to Shivpuri
[2] Scraping: Chose wrong path, focused on liquor: Anna Hazare on Arvind Kejriwal's poll loss
[3] Scraping: Amanatullah Khan slams Rahul Gandhi for campaigning in Okhla to 'defeat' AAP
[4] Scraping: BJP top brass holds hectic parleys to finalise Delhi's next Chief Minister
[5] Scraping: Down but not out: Is Nitish Kumar still the X-factor in Bihar Politics?
[6] Scraping: Nitish Kumar changes party for kursi: Congress targets BJP-JDU alliance
[7] Scraping: BJP seeks dominance in Delhi civic body following Assembly success
[8] Scraping: RJD leader pitches Tejashwi Yadav as chief ministerial face ahead of meet today
[9] Scraping: Is Kanhaiya Kumar stealing Tejashwi Yadav's thunder on jobs, migration in Bihar?
[10] Scraping: BJP considers candidates for Delhi Chief Minister's post after win
[11] Scraping: BJP to release multiple CAG reports in upcoming Delhi assembly

##NDTV - SCRAPING AND ANALYSIS

In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import tempfile
from transformers import pipeline

# NLP Pipelines
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
sentiment_analyzer = pipeline("sentiment-analysis")
bias_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Focus keywords for Delhi and Bihar elections
KEYWORDS = [
    "delhi election", "delhi polls", "aap", "kejriwal", "delhi mla",
    "bihar election", "bihar polls", "tejashwi", "nitish", "bjp", "rjd", "jdu",
    "lok sabha", "2024", "campaign", "manifesto", "seat sharing", "voting"
]

# Selenium Setup
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument(f"--user-data-dir={tempfile.mkdtemp()}")
driver = webdriver.Chrome(options=options)

def is_target_region(text):
    return any(keyword in text.lower() for keyword in KEYWORDS)

def get_articles():
    url = "https://www.ndtv.com/elections"
    print(f"[+] Visiting: {url}")
    driver.get(url)
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    links = []

    for tag in soup.find_all("a", class_="crd_lnk", href=True):
        title = tag.get_text(strip=True)
        href = tag["href"]
        if title and is_target_region(title):
            links.append((title, href))

    return list(set(links))

def extract_article_content(url):
    try:
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        paragraphs = soup.find_all("p")
        content = " ".join(p.get_text(strip=True) for p in paragraphs if len(p.text.strip()) > 30)
        return content
    except Exception as e:
        print(f"[-] Error scraping {url}: {e}")
        return ""

def analyze_text(text):
    try:
        summary = summarizer(text[:1024])[0]["summary_text"]
        sentiment_result = sentiment_analyzer(summary)[0]
        sentiment = sentiment_result["label"]
        sentiment_score = round(sentiment_result["score"], 2)

        bias_result = bias_classifier(summary, candidate_labels=["pro-government", "anti-government", "neutral"])
        bias = bias_result["labels"][0]
        bias_score = round(bias_result["scores"][0], 2)

        return summary, sentiment, sentiment_score, bias, bias_score
    except Exception as e:
        print(f"[-] NLP error: {e}")
        return "Summary failed", "UNKNOWN", 0.0, "UNKNOWN", 0.0

def main():
    articles = get_articles()
    print(f"[+] Found {len(articles)} Delhi/Bihar election-related articles")

    data = []
    for i, (title, url) in enumerate(articles[:15]):
        print(f"[{i+1}] Scraping: {title}")
        content = extract_article_content(url)
        if content:
            summary, sentiment, senti_score, bias, bias_score = analyze_text(content)
            data.append({
                "title": title,
                "url": url,
                "content": content,
                "summary": summary,
                "sentiment": sentiment,
                "sentiment_score": senti_score,
                "bias": bias,
                "bias_score": bias_score,
                "source": "NDTV"
            })
        time.sleep(random.uniform(1.5, 2.5))

    df = pd.DataFrame(data)
    df.to_csv("ndtv_delhi_bihar_election.csv", index=False)
    print(f"[✓] Saved {len(df)} articles to ndtv_delhi_bihar_election.csv")

if __name__ == "__main__":
    main()
    driver.quit()


Device set to use cpu
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
Device set to use cpu


[+] Visiting: https://www.ndtv.com/elections
[+] Found 15 Delhi/Bihar election-related articles
[1] Scraping: "Governance Is Not Theatrics": PM's Dig At AAP After BJP Wins Delhi
[2] Scraping: Delhi Oath Ceremony On Thursday, BJP Yet To Name Chief Minister
[3] Scraping: "AAP Was Finished When...": Ex Leader Ashutosh's Post After Delhi Drubbing
[4] Scraping: Video: AAP's Office Doors Shut As Arvind Kejriwal, Manish Sisodia Lose Delhi
[5] Scraping: "Was Doing Good Job, But...": Anna Hazare On Arvind Kejriwal's Poll Defeat
[6] Scraping: "2026MeinBengalKi Baari": Mamata Banerjee Warned After BJP's Delhi Win
[7] Scraping: AAP's Saurabh Bharadwaj Turns YouTuber After Delhi Election Loss
[8] Scraping: "Rule Of Lies Has Ended": Amit Shah On BJP's Delhi Victory
[9] Scraping: AAP To Replace Chief Minister In Punjab? What Bhagwant Mann Said
[10] Scraping: How AAP-Congress Feud Split Opposition Votes
[11] Scraping: AAP vs Congress In Punjab Over '30 MLAs In Touch With Us' Claim
[12] Scraping: Congr

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import tempfile
from transformers import pipeline

# Load NLP models
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
sentiment_analyzer = pipeline("sentiment-analysis")

def classify_bias(text):
    text = text.lower()
    pro_gov = any(word in text for word in ["modi", "bjp", "nda", "development"])
    anti_gov = any(word in text for word in ["scam", "failure", "protest", "resign", "criticism", "controversy"])
    if pro_gov and not anti_gov:
        return "PRO-GOVERNMENT"
    elif anti_gov and not pro_gov:
        return "ANTI-GOVERNMENT"
    elif pro_gov and anti_gov:
        return "MIXED"
    return "NEUTRAL"

# Keywords
KEYWORDS = ["delhi election", "bihar election", "2024", "lok sabha", "nda", "india bloc", "upa", "bjp", "congress"]

# Selenium setup
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument(f"--user-data-dir={tempfile.mkdtemp()}")
driver = webdriver.Chrome(options=options)

# Fetch TOI Delhi and Bihar election articles
def get_toi_articles():
    urls = [
        "https://timesofindia.indiatimes.com/elections/assembly-elections/delhi",
        "https://timesofindia.indiatimes.com/elections/assembly-elections/bihar"
    ]
    articles = []
    for url in urls:
        driver.get(url)
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        tags = soup.find_all("a", class_="SgKCA", href=True)
        for tag in tags:
            title = tag.get_text(strip=True)
            link = tag['href']
            if any(k in title.lower() for k in KEYWORDS):
                if not link.startswith("http"):
                    link = "https://timesofindia.indiatimes.com" + link
                articles.append((title, link))
            if len(articles) >= 15:
                return articles
    return articles

# Extract article content
def extract_article(url):
    try:
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        paragraphs = soup.find_all("div", class_="Normal")
        if not paragraphs:
            paragraphs = soup.find_all("p")
        content = " ".join(p.get_text(strip=True) for p in paragraphs if len(p.text.strip()) > 30)
        return content
    except Exception as e:
        print(f"[-] Failed to extract {url}: {e}")
        return ""

# Main
def main():
    print("[+] Scraping Times of India for Delhi & Bihar election news")
    articles = get_toi_articles()
    print(f"[+] Found {len(articles)} relevant articles")

    records = []
    for i, (title, url) in enumerate(articles):
        print(f"[{i+1}] {title}")
        content = extract_article(url)
        if content:
            summary = summarizer(content[:1024])[0]['summary_text']
            sentiment_result = sentiment_analyzer(summary)[0]
            sentiment = sentiment_result['label']
            score = round(sentiment_result['score'], 2)
            bias = classify_bias(summary)
            records.append({
                "title": title,
                "url": url,
                "summary": summary,
                "sentiment": sentiment,
                "confidence": score,
                "bias": bias,
                "content": content,
                "source": "TOI"
            })
        time.sleep(random.uniform(1.5, 2.5))

    df = pd.DataFrame(records)
    df.to_csv("toi_delhi_bihar_elections.csv", index=False)
    print("[✓] Data saved to toi_delhi_bihar_elections.csv")

if __name__ == "__main__":
    main()
    driver.quit()


Device set to use cpu
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[+] Scraping Times of India for Delhi & Bihar election news
[+] Found 15 relevant articles
[1] All NCR states now have BJP governments, will give infra a big push, says PM Modi


Your max_length is set to 142, but your input_length is only 106. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=53)


[2] Will work as constructive opposition: AAP; BJP thanks PM
[3] Delhi elections: Four reserved seats shed reservations
[4] In BJP's Delhi win, a booster for Nayab Singh Saini, Manohar Lal Khattar
[5] House of controversy: Delhi BJP says CM won't occupy 'Sheesh Mahal'


Your max_length is set to 142, but your input_length is only 106. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=53)


[6] Sirsa win weakens SAD in Delhi, BJP gains ground among Sikh voters
[7] Delhi elections: Debutants taste nail-biters and commanding leads


Your max_length is set to 142, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


[8] Delhi elections: In a rare four-cornered fight, Amanatullah on top in Okhla
[9] Delhi election results: 'Saffrun' continues in Rohini as Vijender Gupta secures facile hat-trick
[10] Bihar: Opposition boycotts swearing-in, says it’s betrayal of mandate
[11] Bihar: LJP chief congratulates Nitish Kumar, BJP
[12] Bihar: Congress flays Shivanand Tiwari, RJD distances from his statement
[13] BJP gets 2 deputy CMs as diminished Nitish takes oath
[14] Nitish Kumar BJP-nominated CM, tired and politically 'belittled': Prashant Kishor
[15] NDA family will work together for progress of Bihar: PM Modi
[✓] Data saved to toi_delhi_bihar_elections.csv
